# Notebook 17: Calibration and Reliability

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_4_Uncertainty_Estimation_and_Validation/notebooks/17_Calibration_and_Reliability.ipynb)

**Session 4: Uncertainty Estimation and Validation**  
**Duration:** 20 minutes

## Learning Objectives
- Understand model calibration and why it matters
- Learn to measure calibration using reliability diagrams and ECE
- Implement calibration methods (Temperature Scaling, Platt Scaling)
- Apply calibration to AV perception tasks
- Connect calibration to safety decision-making

---

## Introduction

**The Calibration Problem:**

A model predicts "pedestrian" with 90% confidence. What does this mean?
- **Calibrated model:** 90% confidence → 90% chance it's actually a pedestrian
- **Miscalibrated model:** 90% confidence → Could be 50% or 99% actual accuracy!

**Modern neural networks are often overconfident** (Guo et al., 2017)
- They achieve high accuracy but poor calibration
- This is dangerous for AVs: overconfidence → risky decisions

**Why calibration matters for AVs:**
- Decision thresholds require calibrated probabilities
- Safety requires knowing when the model is uncertain
- ISO 26262/21448 require understanding system limitations

In [ ]:
# Setup
!pip install -q torch torchvision matplotlib seaborn numpy scipy scikit-learn netcal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from scipy.optimize import minimize

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Understanding Calibration

### 1.1 Definition

**Perfect Calibration:** For all predictions with confidence p, exactly p fraction are correct.

Mathematically:
```
P(Y = ŷ | P(ŷ) = p) = p
```

**Example:**
- Model makes 100 predictions with 80% confidence
- If calibrated: exactly 80 of them should be correct
- If overconfident: fewer than 80 are correct
- If underconfident: more than 80 are correct

### 1.2 Reliability Diagram

The primary tool for visualizing calibration.

**How it works:**
1. Bin predictions by confidence (e.g., [0-0.1, 0.1-0.2, ..., 0.9-1.0])
2. For each bin, compute average confidence and actual accuracy
3. Plot: x-axis = confidence, y-axis = accuracy
4. Perfect calibration = diagonal line

In [ ]:
def reliability_diagram(y_true, y_prob, n_bins=10, title="Reliability Diagram"):
    """
    Plot reliability diagram.
    
    Args:
        y_true: True labels (binary)
        y_prob: Predicted probabilities for positive class
        n_bins: Number of bins
        title: Plot title
    """
    # Bin edges
    bin_edges = np.linspace(0, 1, n_bins + 1)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Assign predictions to bins
    bin_indices = np.digitize(y_prob, bin_edges[1:-1])
    
    # Compute accuracy and average confidence per bin
    bin_accuracies = []
    bin_confidences = []
    bin_counts = []
    
    for i in range(n_bins):
        mask = bin_indices == i
        if mask.sum() > 0:
            bin_accuracy = y_true[mask].mean()
            bin_confidence = y_prob[mask].mean()
            bin_count = mask.sum()
        else:
            bin_accuracy = 0
            bin_confidence = bin_centers[i]
            bin_count = 0
        
        bin_accuracies.append(bin_accuracy)
        bin_confidences.append(bin_confidence)
        bin_counts.append(bin_count)
    
    bin_accuracies = np.array(bin_accuracies)
    bin_confidences = np.array(bin_confidences)
    bin_counts = np.array(bin_counts)
    
    # Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Reliability diagram
    ax1.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect calibration')
    
    # Bar chart with gap showing calibration error
    gap = bin_confidences - bin_accuracies
    colors = ['red' if g > 0 else 'green' for g in gap]
    
    # Plot bars
    for i, (conf, acc, count) in enumerate(zip(bin_confidences, bin_accuracies, bin_counts)):
        if count > 0:
            # Bar showing actual accuracy
            ax1.bar(conf, acc, width=0.08, alpha=0.6, color='blue', 
                   edgecolor='black', linewidth=1.5)
            # Gap showing miscalibration
            if gap[i] > 0:  # Overconfident
                ax1.bar(conf, gap[i], bottom=acc, width=0.08, alpha=0.4, 
                       color='red', edgecolor='black', linewidth=1.5)
            else:  # Underconfident
                ax1.bar(conf, -gap[i], bottom=conf, width=0.08, alpha=0.4,
                       color='green', edgecolor='black', linewidth=1.5)
    
    # Points showing bin centers
    valid_mask = bin_counts > 0
    ax1.scatter(bin_confidences[valid_mask], bin_accuracies[valid_mask], 
               s=bin_counts[valid_mask]*2, c='blue', alpha=0.8, 
               edgecolors='black', linewidth=2, zorder=5, label='Actual accuracy')
    
    ax1.set_xlabel('Confidence (Predicted Probability)', fontsize=12)
    ax1.set_ylabel('Accuracy (Fraction Correct)', fontsize=12)
    ax1.set_title(title, fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim([0, 1])
    ax1.set_ylim([0, 1])
    
    # Add text annotations
    ax1.text(0.7, 0.2, 'Overconfident\n(Red gap)', fontsize=10, 
            bbox=dict(boxstyle='round', facecolor='red', alpha=0.3))
    ax1.text(0.2, 0.7, 'Underconfident\n(Green gap)', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='green', alpha=0.3))
    
    # Histogram of predictions
    ax2.bar(bin_centers, bin_counts, width=0.08, alpha=0.7, 
           color='steelblue', edgecolor='black', linewidth=1.5)
    ax2.set_xlabel('Confidence', fontsize=12)
    ax2.set_ylabel('Number of Predictions', fontsize=12)
    ax2.set_title('Prediction Distribution', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.set_xlim([0, 1])
    
    plt.tight_layout()
    plt.show()
    
    return bin_accuracies, bin_confidences, bin_counts

print("Reliability diagram function defined!")

## 2. Expected Calibration Error (ECE)

**ECE:** Single scalar metric for calibration quality.

**Formula:**
```
ECE = Σ (|Bₘ| / N) |acc(Bₘ) - conf(Bₘ)|
```

Where:
- Bₘ = predictions in bin m
- |Bₘ| = number of predictions in bin m
- N = total number of predictions
- acc(Bₘ) = accuracy in bin m
- conf(Bₘ) = average confidence in bin m

**Interpretation:**
- ECE = 0: Perfect calibration
- ECE = 0.1: On average, 10% calibration error
- Lower is better

In [ ]:
def expected_calibration_error(y_true, y_prob, n_bins=10):
    """
    Compute Expected Calibration Error (ECE).
    
    Args:
        y_true: True labels (binary)
        y_prob: Predicted probabilities for positive class
        n_bins: Number of bins
        
    Returns:
        ece: Expected Calibration Error
    """
    bin_edges = np.linspace(0, 1, n_bins + 1)
    bin_indices = np.digitize(y_prob, bin_edges[1:-1])
    
    ece = 0.0
    for i in range(n_bins):
        mask = bin_indices == i
        if mask.sum() > 0:
            bin_accuracy = y_true[mask].mean()
            bin_confidence = y_prob[mask].mean()
            bin_size = mask.sum()
            
            ece += (bin_size / len(y_true)) * abs(bin_accuracy - bin_confidence)
    
    return ece

print("ECE function defined!")

## 3. Creating a Miscalibrated Model

Let's train a model and observe its calibration.

In [ ]:
# Create dataset
X, y = make_classification(n_samples=2000, n_features=20, n_informative=15,
                          n_redundant=5, random_state=42)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

In [ ]:
# Define neural network
class BinaryClassifier(nn.Module):
    """Simple binary classifier."""
    
    def __init__(self, input_dim=20, hidden_dim=100):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Train model
model = BinaryClassifier(input_dim=20, hidden_dim=100).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)
y_val_tensor = torch.FloatTensor(y_val).unsqueeze(1).to(device)

# Training loop
train_losses = []
val_losses = []

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_losses.append(val_loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/100, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

# Plot training
plt.figure(figsize=(10, 4))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Get predictions on test set
model.eval()
X_test_tensor = torch.FloatTensor(X_test).to(device)

with torch.no_grad():
    logits = model(X_test_tensor).cpu().numpy().flatten()
    probs_uncalibrated = torch.sigmoid(torch.FloatTensor(logits)).numpy()

# Compute accuracy
predictions = (probs_uncalibrated > 0.5).astype(int)
accuracy = (predictions == y_test).mean()

# Compute ECE
ece_before = expected_calibration_error(y_test, probs_uncalibrated)

print(f"\nTest Accuracy: {accuracy:.4f}")
print(f"ECE (before calibration): {ece_before:.4f}")

# Plot reliability diagram
print("\n⚠️ BEFORE CALIBRATION:")
_ = reliability_diagram(y_test, probs_uncalibrated, n_bins=10, 
                       title="Reliability Diagram - UNCALIBRATED Model")

## 4. Calibration Methods

### 4.1 Temperature Scaling

**Idea:** Scale the logits by a temperature parameter T.

**Formula:**
```
Original:  p = softmax(z)
Calibrated: p = softmax(z / T)
```

**Properties:**
- T > 1: Softens probabilities (less confident)
- T < 1: Sharpens probabilities (more confident)
- T = 1: No change
- Optimized on validation set to minimize NLL or ECE

**Advantages:**
- ✅ Single parameter (simple!)
- ✅ Preserves accuracy (predictions don't change)
- ✅ Fast and effective
- ✅ State-of-the-art for neural networks

In [ ]:
class TemperatureScaling:
    """
    Temperature scaling for calibration.
    """
    
    def __init__(self):
        self.temperature = 1.0
    
    def fit(self, logits, labels, max_iter=50):
        """
        Learn optimal temperature on validation set.
        
        Args:
            logits: Model logits (before sigmoid)
            labels: True labels
        """
        logits = torch.FloatTensor(logits).unsqueeze(1)
        labels = torch.FloatTensor(labels).unsqueeze(1)
        
        # Optimize temperature to minimize negative log likelihood
        def nll_criterion(T):
            scaled_logits = logits / T
            loss = F.binary_cross_entropy_with_logits(scaled_logits, labels)
            return loss.item()
        
        # Find optimal temperature
        result = minimize(nll_criterion, x0=1.0, method='L-BFGS-B', 
                         bounds=[(0.1, 10.0)], options={'maxiter': max_iter})
        
        self.temperature = result.x[0]
        print(f"Optimal temperature: {self.temperature:.4f}")
        
        return self
    
    def predict_proba(self, logits):
        """
        Get calibrated probabilities.
        
        Args:
            logits: Model logits
            
        Returns:
            Calibrated probabilities
        """
        logits = torch.FloatTensor(logits)
        scaled_logits = logits / self.temperature
        probs = torch.sigmoid(scaled_logits).numpy()
        return probs

print("Temperature Scaling class defined!")

In [ ]:
# Apply temperature scaling
# Get validation logits
with torch.no_grad():
    val_logits = model(X_val_tensor).cpu().numpy().flatten()

# Fit temperature scaling on validation set
temp_scaler = TemperatureScaling()
temp_scaler.fit(val_logits, y_val)

# Get calibrated probabilities on test set
probs_temp_scaled = temp_scaler.predict_proba(logits)

# Compute ECE
ece_temp = expected_calibration_error(y_test, probs_temp_scaled)

print(f"\nECE after temperature scaling: {ece_temp:.4f}")
print(f"Improvement: {(ece_before - ece_temp) / ece_before * 100:.1f}%")

# Plot reliability diagram
print("\n✅ AFTER TEMPERATURE SCALING:")
_ = reliability_diagram(y_test, probs_temp_scaled, n_bins=10,
                       title="Reliability Diagram - TEMPERATURE SCALED")

### 4.2 Platt Scaling

**Idea:** Fit a logistic regression on top of model outputs.

**Formula:**
```
p_calibrated = sigmoid(A * z + B)
```

Where A and B are learned from validation data.

**Properties:**
- More flexible than temperature scaling (2 parameters)
- Can handle both over- and under-confidence
- Originally designed for SVMs, works for neural networks too

In [ ]:
class PlattScaling:
    """
    Platt scaling for calibration.
    """
    
    def __init__(self):
        self.lr = LogisticRegression()
    
    def fit(self, logits, labels):
        """
        Fit Platt scaling on validation set.
        
        Args:
            logits: Model logits
            labels: True labels
        """
        self.lr.fit(logits.reshape(-1, 1), labels)
        print(f"Platt scaling parameters: A={self.lr.coef_[0][0]:.4f}, B={self.lr.intercept_[0]:.4f}")
        return self
    
    def predict_proba(self, logits):
        """
        Get calibrated probabilities.
        """
        probs = self.lr.predict_proba(logits.reshape(-1, 1))[:, 1]
        return probs

# Apply Platt scaling
platt_scaler = PlattScaling()
platt_scaler.fit(val_logits, y_val)

# Get calibrated probabilities
probs_platt = platt_scaler.predict_proba(logits)

# Compute ECE
ece_platt = expected_calibration_error(y_test, probs_platt)

print(f"\nECE after Platt scaling: {ece_platt:.4f}")
print(f"Improvement: {(ece_before - ece_platt) / ece_before * 100:.1f}%")

# Plot
print("\n✅ AFTER PLATT SCALING:")
_ = reliability_diagram(y_test, probs_platt, n_bins=10,
                       title="Reliability Diagram - PLATT SCALED")

### 4.3 Isotonic Regression

**Idea:** Learn a non-parametric monotonic mapping.

**Properties:**
- Most flexible (non-parametric)
- Can fit complex calibration curves
- Risk of overfitting with small validation sets
- Requires more validation data than temperature scaling

In [ ]:
class IsotonicCalibration:
    """
    Isotonic regression for calibration.
    """
    
    def __init__(self):
        self.iso_reg = IsotonicRegression(out_of_bounds='clip')
    
    def fit(self, probs, labels):
        """
        Fit isotonic regression.
        
        Args:
            probs: Predicted probabilities (after sigmoid)
            labels: True labels
        """
        self.iso_reg.fit(probs, labels)
        return self
    
    def predict_proba(self, probs):
        """
        Get calibrated probabilities.
        """
        return self.iso_reg.predict(probs)

# Apply isotonic regression
# Need probabilities (not logits) for isotonic regression
val_probs_uncalib = torch.sigmoid(torch.FloatTensor(val_logits)).numpy()

iso_scaler = IsotonicCalibration()
iso_scaler.fit(val_probs_uncalib, y_val)

# Get calibrated probabilities
probs_isotonic = iso_scaler.predict_proba(probs_uncalibrated)

# Compute ECE
ece_iso = expected_calibration_error(y_test, probs_isotonic)

print(f"\nECE after isotonic regression: {ece_iso:.4f}")
print(f"Improvement: {(ece_before - ece_iso) / ece_before * 100:.1f}%")

# Plot
print("\n✅ AFTER ISOTONIC REGRESSION:")
_ = reliability_diagram(y_test, probs_isotonic, n_bins=10,
                       title="Reliability Diagram - ISOTONIC REGRESSION")

## 5. Comparison of Calibration Methods

In [ ]:
def compare_calibration_methods():
    """Compare all calibration methods."""
    
    methods = {
        'Uncalibrated': probs_uncalibrated,
        'Temperature Scaling': probs_temp_scaled,
        'Platt Scaling': probs_platt,
        'Isotonic Regression': probs_isotonic
    }
    
    # Compute ECE for all
    eces = {name: expected_calibration_error(y_test, probs) 
            for name, probs in methods.items()}
    
    # Plot comparison
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    for ax, (name, probs) in zip(axes, methods.items()):
        # Reliability curve
        bin_edges = np.linspace(0, 1, 11)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        bin_indices = np.digitize(probs, bin_edges[1:-1])
        
        bin_accs = []
        bin_confs = []
        bin_counts = []
        
        for i in range(10):
            mask = bin_indices == i
            if mask.sum() > 0:
                bin_accs.append(y_test[mask].mean())
                bin_confs.append(probs[mask].mean())
                bin_counts.append(mask.sum())
        
        # Plot perfect calibration
        ax.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfect')
        
        # Plot actual calibration
        if len(bin_confs) > 0:
            ax.plot(bin_confs, bin_accs, 'o-', linewidth=3, markersize=10,
                   label=f'Actual (ECE={eces[name]:.3f})')
        
        # Color based on ECE
        if eces[name] < 0.05:
            color = 'lightgreen'
        elif eces[name] < 0.10:
            color = 'yellow'
        else:
            color = 'lightcoral'
        ax.set_facecolor(color)
        
        ax.set_xlabel('Confidence', fontsize=11)
        ax.set_ylabel('Accuracy', fontsize=11)
        ax.set_title(f'{name}\nECE = {eces[name]:.4f}', 
                    fontsize=13, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_xlim([0, 1])
        ax.set_ylim([0, 1])
    
    plt.tight_layout()
    plt.show()
    
    # Bar chart comparison
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # ECE comparison
    names = list(eces.keys())
    values = list(eces.values())
    colors_bar = ['red', 'lightgreen', 'lightgreen', 'green']
    
    bars = ax1.bar(names, values, color=colors_bar, alpha=0.7, 
                   edgecolor='black', linewidth=2)
    ax1.set_ylabel('Expected Calibration Error', fontsize=12)
    ax1.set_title('Calibration Method Comparison', fontsize=14, fontweight='bold')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Add values on bars
    for bar, val in zip(bars, values):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.4f}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Improvement over baseline
    baseline = eces['Uncalibrated']
    improvements = [(baseline - ece) / baseline * 100 
                   for ece in list(eces.values())[1:]]
    
    bars2 = ax2.bar(names[1:], improvements, color=colors_bar[1:], 
                   alpha=0.7, edgecolor='black', linewidth=2)
    ax2.set_ylabel('Improvement over Uncalibrated (%)', fontsize=12)
    ax2.set_title('ECE Reduction', fontsize=14, fontweight='bold')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3, axis='y')
    
    for bar, val in zip(bars2, improvements):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.1f}%',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print("\n" + "="*70)
    print("CALIBRATION METHODS COMPARISON")
    print("="*70)
    print(f"\n{'Method':<25} {'ECE':<10} {'Improvement':<15} {'Parameters'}")
    print("-"*70)
    
    params = ['0', '1', '2', 'Non-parametric']
    for (name, ece), param in zip(eces.items(), params):
        if name == 'Uncalibrated':
            improvement = '-'
        else:
            improvement = f"{(baseline - ece) / baseline * 100:.1f}%"
        print(f"{name:<25} {ece:<10.4f} {improvement:<15} {param}")
    
    print("\n" + "="*70)
    print("\n🎯 Recommendations:")
    print("\n1. Temperature Scaling:")
    print("   ✅ Best choice for most applications")
    print("   ✅ Simple, effective, preserves accuracy")
    print("   ✅ Works well with small validation sets")
    print("\n2. Platt Scaling:")
    print("   ✅ Good alternative, slightly more flexible")
    print("   ⚠️  Two parameters (slightly more risk of overfitting)")
    print("\n3. Isotonic Regression:")
    print("   ✅ Most flexible, can handle complex miscalibration")
    print("   ⚠️  Needs larger validation set")
    print("   ⚠️  Risk of overfitting")
    print("\n💡 For AV production: Use Temperature Scaling")
    print("   Simple, robust, well-understood, industry standard")

compare_calibration_methods()

## 6. Why Calibration Matters for AV Safety

### 6.1 Decision Thresholds

**Problem:** AVs need to make binary decisions based on probabilities.

Example: Emergency braking for pedestrian detection
- If P(pedestrian) > threshold → BRAKE
- If P(pedestrian) ≤ threshold → Continue

**With miscalibrated model:**
- Overconfident model: Threshold seems safe but actually risky
- False sense of security

**With calibrated model:**
- Threshold has clear probabilistic meaning
- Can reason about false positive/negative rates

### 6.2 Safety Standards

**ISO 26262:**
- Requires understanding system behavior in all scenarios
- Calibrated probabilities help define confidence levels

**ISO 21448 (SOTIF):**
- Requires knowing when system is uncertain
- Calibration ensures uncertainty estimates are meaningful

In [ ]:
def demonstrate_decision_threshold_impact():
    """
    Demonstrate how calibration affects decision-making.
    """
    
    # Decision threshold
    threshold = 0.7
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    methods = [
        ('Uncalibrated', probs_uncalibrated),
        ('Temperature Scaled', probs_temp_scaled),
        ('Platt Scaled', probs_platt),
        ('Isotonic', probs_isotonic)
    ]
    
    for ax, (name, probs) in zip(axes.flatten(), methods):
        # Classify based on threshold
        decisions = (probs > threshold).astype(int)
        
        # Compute metrics
        tp = ((decisions == 1) & (y_test == 1)).sum()
        fp = ((decisions == 1) & (y_test == 0)).sum()
        tn = ((decisions == 0) & (y_test == 0)).sum()
        fn = ((decisions == 0) & (y_test == 1)).sum()
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        
        # For safety: False Negative Rate is critical!
        fnr = fn / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (tn + fp) if (tn + fp) > 0 else 0
        
        # Plot confusion matrix style
        conf_matrix = np.array([[tn, fp], [fn, tp]])
        
        im = ax.imshow(conf_matrix, cmap='RdYlGn', alpha=0.7)
        
        # Add text
        for i in range(2):
            for j in range(2):
                text = ax.text(j, i, f'{conf_matrix[i, j]}',
                             ha="center", va="center", color="black",
                             fontsize=20, fontweight='bold')
        
        ax.set_xticks([0, 1])
        ax.set_yticks([0, 1])
        ax.set_xticklabels(['Pred Neg', 'Pred Pos'])
        ax.set_yticklabels(['True Neg', 'True Pos'])
        
        # Safety color coding
        ece = expected_calibration_error(y_test, probs)
        if ece < 0.05:
            border_color = 'green'
        elif ece < 0.10:
            border_color = 'orange'
        else:
            border_color = 'red'
        
        for spine in ax.spines.values():
            spine.set_edgecolor(border_color)
            spine.set_linewidth(4)
        
        ax.set_title(f"{name}\n"
                    f"ECE={ece:.3f} | FNR={fnr:.3f} (⚠️ Safety!)\n"
                    f"Precision={precision:.3f} | Recall={recall:.3f}",
                    fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🎯 Decision Threshold Analysis (threshold = {threshold})")
    print("="*80)
    print(f"\n{'Method':<20} {'ECE':<8} {'FNR':<8} {'FPR':<8} {'Precision':<10} {'Recall':<8}")
    print("-"*80)
    
    for name, probs in methods:
        decisions = (probs > threshold).astype(int)
        tp = ((decisions == 1) & (y_test == 1)).sum()
        fp = ((decisions == 1) & (y_test == 0)).sum()
        tn = ((decisions == 0) & (y_test == 0)).sum()
        fn = ((decisions == 0) & (y_test == 1)).sum()
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        fnr = fn / (tp + fn) if (tp + fn) > 0 else 0
        fpr = fp / (tn + fp) if (tn + fp) > 0 else 0
        ece = expected_calibration_error(y_test, probs)
        
        print(f"{name:<20} {ece:<8.3f} {fnr:<8.3f} {fpr:<8.3f} {precision:<10.3f} {recall:<8.3f}")
    
    print("\n" + "="*80)
    print("\n⚠️  Key Insight for AV Safety:")
    print("   - FNR (False Negative Rate) is CRITICAL for pedestrian detection")
    print("   - Missing a pedestrian = potential fatality")
    print("   - Calibration helps us set thresholds with known FNR")
    print("   - Better calibration → better safety-performance trade-off")

demonstrate_decision_threshold_impact()

## Summary and Key Takeaways

### What We Learned

1. **Calibration Definition:**
   - Confidence should match actual accuracy
   - Modern neural networks are often overconfident
   - Measured using reliability diagrams and ECE

2. **Calibration Methods:**
   - **Temperature Scaling:** Simple, effective, industry standard (1 parameter)
   - **Platt Scaling:** More flexible (2 parameters)
   - **Isotonic Regression:** Most flexible (non-parametric)

3. **Why It Matters for AVs:**
   - Decision thresholds require calibrated probabilities
   - Safety standards (ISO 26262, SOTIF) require understanding uncertainty
   - False negatives in pedestrian detection = safety-critical

4. **Best Practices:**
   - Always check calibration on validation set
   - Temperature scaling is recommended for most cases
   - Calibration is a post-processing step (doesn't change accuracy)

### Next Steps

- **Notebook 18:** Learn validation and testing strategies for AV perception

---

## Interactive Exercise

**Try this:**
1. Change the number of bins in reliability diagram (try 5, 15, 20)
2. Try different decision thresholds (0.5, 0.8, 0.9)
3. Train the model longer - does it become more miscalibrated?

**Challenge:**
- Implement calibration for a multi-class problem (3+ classes)
- How does temperature scaling work with softmax?

**Discussion:**
- For pedestrian detection, would you prefer:
  - High precision (few false alarms)?
  - High recall (few missed pedestrians)?
- How does calibration help you make this trade-off?